In [ ]:
from huggingface_hub import login


/media/sudip/linux-extra/code/llm-fine-tune/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install -q -U bitsandbytes transformers accelerate peft trl datasets gradio

^C
ERROR: Operation cancelled by user


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load dataset (replace with your dataset name)
dataset = load_dataset("soniawmeyer/reddit-travel-QA-finetuning")

def format_data(example):
    context = f"Title: {example['title']}\nselftext: {example['selftext']}\nComments: {example['comments']}"
    return {
        "text": tokenizer.apply_chat_template([
            {"role": "system", "content": "You are a helpful travel assistant."},
            {"role": "user", "content": context},
            {"role": "assistant", "content": example["falcon_summary"]}
        ], tokenize=False)
    }

# Tokenize and format
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token
formatted_data = dataset.map(format_data, remove_columns=dataset["train"].column_names)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

reddit_dot_scores_quality.csv:   0%|          | 0.00/54.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10482 [00:00<?, ? examples/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Map:   0%|          | 0/10482 [00:00<?, ? examples/s]

In [3]:
from peft import LoraConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    quantization_config=bnb_config,
    device_map="auto"
)

# LoRA config
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments

torch.cuda.empty_cache()

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    max_steps=500,
    logging_steps=10,
    fp16=True
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_data["train"],
    # dataset_text_field="text",
    # max_seq_length=1024,
    peft_config=peft_config
)
torch.cuda.empty_cache()
trainer.train()

# save the model in google drive
from google.colab import drive
drive.mount('/content/drive')
trainer.save_model("/content/drive/MyDrive/travel_assistant_model")


Adding EOS to train dataset:   0%|          | 0/10482 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10482 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2581 > 2048). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/10482 [00:00<?, ? examples/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sudip-bairagi (sudip-bairagi-tech-mahindra) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.007100
20,2.826600
30,2.889300
40,2.832700
50,2.830700
60,2.790400
70,2.803600
80,2.805400
90,2.753400
100,2.834500


Mounted at /content/drive


In [5]:
# load the model

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

from google.colab import drive
drive.mount('/content/drive')

# 1. Load the base model (same as used for training)
base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Replace if you used a different base
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

# 2. Load the fine-tuned LoRA adapters
peft_model_id = "/content/drive/MyDrive/travel_assistant_model"  # Path to your saved model
config = PeftConfig.from_pretrained(peft_model_id)

# Load base model with the same quantization config used during training
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Attach the LoRA adapters to the base model
model = PeftModel.from_pretrained(base_model, peft_model_id)
model.eval()  # Set to evaluation mode

Mounted at /content/drive


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [6]:
# Create function for chat inferencing

def generate_response(query):
    messages = [
        {"role": "system", "content": "You are a travel assistant."},
        {"role": "user", "content": query}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(
        outputs[0][inputs.shape[1]:],
        skip_special_tokens=True
    )
    return response

# Test
print(generate_response("What are the best places to visit in India?"))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|assistant|>
What are the best places to visit in India? The best places to visit in India depend on your interest and personal preferences. Here are some popular options. I suggest you see as many as possible. India has many places to visit depending on your interests and availability of time. Some popular places to visit in India are Delhi, Varanasi, Bangalore, Goa, Mumbai, Jaipur, Chandigarh, Himachal Pradesh, Rajasthan, Kerala, Tamil Nadu, Gujarat, Uttarandhra, East Coast, North East Coast, North West Coast, Andhra Pradesh, Karnataka, Maharashtra, West Coast, Central Coast, South Coast, East Coast, South India, South West Coast, South East Coast, South West Coast, Southwest Coast, South West Coast, Southwest Coast, Southwest Coast, South West Coast. You can also find more travel guides on India travel, like travel maps and itineraries, tips and tricks, and more. India has a lot of attractions and places to visit, so it can be difficult to decide which places to visit first


In [7]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 🧳 Travel Assistant")
    chatbot = gr.Chatbot(height=300)
    msg = gr.Textbox(label="Your Question")
    clear = gr.Button("Clear Chat")

    def respond(message, chat_history):
        response = generate_response(message)
        chat_history.append((message, response))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(share=True)

/tmp/ipython-input-4019646588.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=300)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://337e9f02fd94a485aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
